In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from numpy import random
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import DataLoader
from collections import deque
import numpy as np
import multiprocessing
import time
from numpy import array
import pandas as pd

torch.manual_seed(1)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from sklearn.preprocessing import MinMaxScaler
#rotate a list by given offset, (negative offset, right to left) 
def rotateList(items, offset):
    return np.roll(items,offset)

def secondToMillisecond(val):
    return val*1000

def gaussiantimeseries(limit = 60, interval = 2):
    data = {}
    for t in range(0,limit,interval):
        data[t]=random.normal(loc = random.uniform(1,100), scale =1.5, size =1000).astype(float)
    return data

def createsignal(limit = 60, interval = 2):
    signal1, signal2, offset = [],[],[]
    limit = secondToMillisecond(limit)
    offset = random.randint(-10,10)
    data = gaussiantimeseries(limit, interval)
    for t in data:
        dist = data[t]
        signal1.append(random.choice(dist))
        signal2.append(random.choice(dist))
    return array(signal1).reshape((len(signal1), 1)), array(signal2).reshape((len(signal2), 1)), array(rotateList(signal2,offset)).reshape((len(signal2), 1)), offset

def createsignals(num = 50, limit = 60, interval = 2, normalize = False):
    input_seq = []
    data = []
    for i in range(num):
        sig1, sig2original, sig2, offset = createsignal(limit = limit)
        data.append([sig1, sig2original, sig2, offset])
        train_data = np.hstack((sig1,sig2)).astype(float)
        #train data without normalization
        if not normalize:
            train_data = torch.FloatTensor(train_data)
            input_seq.append((train_data, torch.FloatTensor([offset])))
        else:
            scaler = MinMaxScaler(feature_range=(-1, 1))
            train_data_normalized = scaler.fit_transform(train_data)
            train_data_normalized = torch.FloatTensor(train_data_normalized)
            input_seq.append((train_data_normalized, torch.FloatTensor([offset])))
    return input_seq, data
# print(createsignal())

In [ ]:
signal1,signal2,signal2rotated, offset = createsignal(limit=5, interval = 1)
print(f"signal1 {signal1.T}")
print(f"signal2 {signal2.T}")
print(f"signal2 offset {signal2rotated.T}")
print(f"offset {offset}")
# print(torch.FloatTensor(np.hstack((array(a),array(b)))).shape)

signal1 [[89.19262649  2.3358773   9.61857725 ... 84.84878357 30.380714
  44.24270055]]
signal2 [[91.48853034  1.20354769 11.92008974 ... 87.31665907 28.36057138
  43.17548394]]
signal2 offset [[80.44920154 44.95623505 12.26072353 ... 28.06016582 33.31552582
  58.73089043]]
offset 119


In [ ]:
# df = pd.DataFrame(columns=['signal1', 'signal2original', 'signal2', 'offset'])
_, data = createsignals(num=1, limit =5)
df = pd.DataFrame(data, columns=['signal1', 'signal2original', 'signal2', 'offset'])

In [ ]:
df

,signal1,signal2original,signal2,offset
0,"[[56.819353131940424], [74.51014160171344], [5...","[[55.43100888927905], [74.24918127282967], [54...","[[80.8478539359136], [38.09122806055196], [10....",133


In [ ]:
input = createsignals(num=1, limit=10, normalize=True)
print(input)

([(tensor([[ 0.7482, -0.9026],
        [-0.6746, -0.6717],
        [ 0.2344,  0.6061],
        ...,
        [-0.5235,  0.2135],
        [ 0.3200, -0.8049],
        [-0.0990, -0.3927]]), tensor([86.]))], [[array([[90.05913333],
       [14.49301148],
       [62.7694761 ],
       ...,
       [22.51565071],
       [67.31340822],
       [45.06147924]]), array([[90.62068456],
       [12.39322878],
       [62.97569684],
       ...,
       [24.30280675],
       [68.79929354],
       [48.36113186]]), array([[ 2.67455148],
       [14.91622797],
       [82.64365659],
       ...,
       [61.83496475],
       [ 7.85236342],
       [29.70127143]]), 86]])


In [4]:
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return torch.round(predictions[-1])

In [5]:
train_input, train_data = createsignals(num=200, limit = 60, interval = 1, normalize = False)
test_input, test_data = createsignals()

In [4]:
train_df = pd.DataFrame(train_data, columns=['signal1', 'signal2original', 'signal2', 'offset'])

In [5]:
train_df

,signal1,signal2original,signal2,offset
0,"[[60.501353411803976], [40.817234776709114], [...","[[62.18486483098474], [41.69206967575986], [10...","[[69.5969509973684], [78.10088234566591], [9.7...",-4
1,"[[70.03148403965176], [7.108018998611433], [57...","[[69.6401516301628], [9.741321253435855], [60....","[[8.983020923646954], [17.84754335788158], [22...",-4
2,"[[27.055538775234194], [68.04036917634608], [3...","[[25.374517330694598], [65.17815783888048], [3...","[[34.632818039785924], [65.5218462457058], [52...",-2
3,"[[28.119578802406803], [23.324402597932007], [...","[[30.060927062918285], [25.927127484197598], [...","[[86.84072733384275], [3.9823882172694898], [4...",5
4,"[[57.86681305051925], [98.62297395587213], [4....","[[58.71487757510003], [98.54255363094515], [6....","[[4.691113579717762], [70.26746405439948], [75...",9
...,...,...,...,...
495,"[[14.409907334460161], [61.53278476591575], [7...","[[14.220291672527797], [62.3713039685288], [6....","[[14.220291672527797], [62.3713039685288], [6....",0
496,"[[54.219856761660054], [74.21306885103189], [-...","[[52.90809839455389], [72.65179460641902], [3....","[[21.761297627778973], [51.58863551162011], [1...",9
497,"[[81.13087878380476], [50.571169709358344], [4...","[[80.12489381754511], [52.70192769397994], [46...","[[85.54153286333406], [83.5632750404834], [8.2...",4
498,"[[94.14703764956282], [4.597042185312168], [27...","[[98.31412175310123], [4.882788685233131], [24...","[[40.66475618808265], [58.2884372374301], [51....",-8


In [ ]:
train_input[:1]

[(tensor([[50.6323, 18.9472],
          [67.0676, 43.5327],
          [44.4008, 27.8539],
          ...,
          [29.5592, 70.8102],
          [ 1.5655, 71.4079],
          [16.6151, 63.4416]]), tensor([-160.]))]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!mkdir '/content/drive/My Drive/DummyGaussianData'


In [ ]:
train_df.to_csv (r'/content/drive/My Drive/DummyGaussianData/train_df.csv', index = False, header=True)

In [ ]:
dataframe = pd.read_csv('/content/drive/My Drive/DummyGaussianData/train_df.csv')

In [ ]:
dataframe

,signal1,signal2original,signal2,offset
0,[[ 8.43199028]\n [80.63800107]\n [30.66877376]...,[[ 8.58355676]\n [84.20628182]\n [30.95570389]...,[[65.09646881]\n [13.70389671]\n [86.33123282]...,-3.992
1,[[ 3.56463304]\n [91.89861125]\n [ 4.03192954]...,[[ 2.9256586 ]\n [94.24165509]\n [ 3.11135898]...,[[98.52443438]\n [44.46601158]\n [58.17686744]...,4.603
2,[[31.15871225]\n [61.18602927]\n [22.68841393]...,[[32.69972756]\n [64.22961137]\n [24.89498547]...,[[17.45868192]\n [53.3587608 ]\n [42.89900618]...,1.259
3,[[75.96086442]\n [69.61594011]\n [47.15193354]...,[[74.07688803]\n [72.51466571]\n [45.52109198]...,[[53.17478846]\n [47.81145748]\n [20.0333093 ]...,1.582
4,[[18.16368526]\n [62.38101936]\n [80.79783069]...,[[21.05429785]\n [64.0977248 ]\n [82.40463446]...,[[ 5.9125675 ]\n [50.6936321 ]\n [34.17214597]...,1.371
...,...,...,...,...
995,[[14.92367321]\n [20.17069222]\n [62.74599815]...,[[14.42668791]\n [19.34540131]\n [63.15980778]...,[[13.34283837]\n [69.70831292]\n [27.73238767]...,0.829
996,[[50.66982818]\n [64.43296584]\n [30.64579142]...,[[52.45368821]\n [65.48957947]\n [33.95889432]...,[[72.32433904]\n [ 1.47123656]\n [39.23719529]...,0.248
997,[[ 3.70432256]\n [38.1710181 ]\n [46.4965546 ]...,[[ 3.27892359]\n [37.3417518 ]\n [49.66184049]...,[[100.14858666]\n [ 13.1588898 ]\n [ 91.401783...,-1.237
998,[[35.83070232]\n [76.58659062]\n [73.20799945]...,[[37.1162726 ]\n [78.33119897]\n [74.02410157]...,[[55.66910965]\n [15.6203073 ]\n [ 4.42562668]...,-4.215


In [7]:
model = LSTM(input_size=2)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

In [ ]:
epochs = 50
model.train()
for epoch in range(epochs):
    print("Epoch {}/{}".format(epoch+1, epochs))
    print('-' * 10)
    epoch_loss = 0.0
    running_loss = 0.0
    model.zero_grad()
    for seq, labels in train_input:

        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size))
        # seq, labels = seq.to(device), labels.to(device)
        y_pred = model(seq)#.to(device)
        optimizer.zero_grad()
        loss = criterion(y_pred, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += y_pred.shape[0] * loss.item()

        # print statistics
        # running_loss += loss.item()
        # if i % 25 == 1:    
        #     print('[%d] loss: %.3f' %
        #           (i + 1, running_loss / 25))
        #     running_loss = 0.0
    print("Epoch {} Loss {}".format(epoch+1, epoch_loss / len(train_input)))

    # if i%25 == 1:
#     print(f'epoch: {i:3} loss: {loss.item():10.8f}')

# print(f'epoch: {i:3} loss: {loss.item():10.10f}')

Epoch 1/50
----------
Epoch 1 Loss 31.57
Epoch 2/50
----------
Epoch 2 Loss 31.57
Epoch 3/50
----------
Epoch 3 Loss 31.57
Epoch 4/50
----------
Epoch 4 Loss 31.57
Epoch 5/50
----------
Epoch 5 Loss 31.57
Epoch 6/50
----------
Epoch 6 Loss 31.57
Epoch 7/50
----------
Epoch 7 Loss 31.57
Epoch 8/50
----------


In [ ]:
epochs = 1
model.eval()
for epoch in range(epochs):
    print("Epoch {}/{}".format(epoch+1, epochs))
    print('-' * 10)
    epoch_loss = 0.0
    running_loss = 0.0
    # model.zero_grad()
    for seq, labels in test_input:

        # model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
        #                 torch.zeros(1, 1, model.hidden_layer_size))
        # # seq, labels = seq.to(device), labels.to(device)
        y_pred = model(seq)#.to(device)
        print(f"Offset {labels.data[0]}, Prediction {y_pred.data[0]}")
        # optimizer.zero_grad()
        loss = criterion(y_pred, labels)
        # loss.backward()
        # optimizer.step()
        epoch_loss += y_pred.shape[0] * loss.item()

        # print statistics
        # running_loss += loss.item()
        # if i % 25 == 1:    
        #     print('[%d] loss: %.3f' %
        #           (i + 1, running_loss / 25))
        #     running_loss = 0.0
    print("Epoch {} Loss {}".format(epoch+1, epoch_loss / len(train_input)))

    # if i%25 == 1:
#     pr

Epoch 1/1
----------
Offset -1.0, Prediction -1.3300405740737915
Offset 1.0, Prediction 1.00392484664917
Offset 3.0, Prediction 2.5719196796417236
Offset -3.0, Prediction -3.5593316555023193
Offset 3.0, Prediction -1.3332680463790894
Offset -1.0, Prediction -0.802498996257782
Offset -5.0, Prediction -2.7559022903442383
Offset -4.0, Prediction -1.5229063034057617
Offset 0.0, Prediction 0.006116407923400402
Offset 0.0, Prediction 0.018826615065336227
Offset -4.0, Prediction -3.026475667953491
Offset 0.0, Prediction -0.30811968445777893
Offset -5.0, Prediction -4.173523902893066
Offset 2.0, Prediction 1.4957826137542725
Offset 1.0, Prediction 1.166930079460144
Offset -1.0, Prediction -0.7062098383903503
Offset 1.0, Prediction 1.3860684633255005
Offset 0.0, Prediction -0.16058781743049622
Offset 2.0, Prediction 1.7859256267547607
Offset 2.0, Prediction 1.931938886642456
Offset -5.0, Prediction -4.232842922210693
Offset 4.0, Prediction 2.8655223846435547
Offset 2.0, Prediction 2.14780688285